<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

# **California Housing Model Playground Deployment & Competition Creation**

# Objective: Predict median house value for California districts, expressed in hundreds of thousands of dollars

---

**Data**: 1990 Census attributes by Block Group. 
(A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data. A block group typically has a population of 600 to 3,000 people.) 

**Features**
* **MedInc** median income in block group
* **HouseAge** median house age in block group
* **AveRooms** average number of rooms per household
* **AveBedrms** average number of bedrooms per household
* **Population** block group population
* **AveOccup** average number of household members
* **Latitude** block group latitude
* **Longitude** block group longitude

**Target**
*   Median house value for California districts, expressed in hundreds of thousands of dollars ($100,000)

*Source: Sklearn [California Housing Dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)*

## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*

In [ ]:
# install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file (skip this step if you have a file already)
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

AI Modelshare Username:··········
AI Modelshare Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


# **Part 1: Prep Data, Preprocessor, & Model**

In [2]:
from aimodelshare.aws import set_credentials
set_credentials(credential_file = "credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [3]:
# Get deployment data
import sklearn
from sklearn import datasets

dataset = sklearn.datasets.fetch_california_housing(as_frame=True)
X = dataset.data 
y = dataset.target

In [4]:
print(X.shape) 
print(y.shape)

(20640, 8)
(20640,)


In [5]:
# Set up training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_train.columns.tolist())

(16512, 8)
(16512,)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


### **Define Preprocessor** <br>
Preprocess data using sklearn's Column Transformer

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Create the preprocessing pipeline for both numeric data.
numeric_features=X.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# final preprocessor object set up with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features)])

#Fit your preprocessor object
preprocess=preprocessor.fit(X_train) 

In [7]:
# Write function to transform data with preprocessor

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [8]:
# Check shape for keras input:
preprocessor(X_train).shape 

(16512, 8)

In [9]:
#Save Example Data 
example_data = X_train[50:55]
example_data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
4420,2.4038,46.0,3.612654,1.084877,2371.0,3.658951,34.07,-118.26
1413,4.4750,43.0,5.635659,0.906977,683.0,2.647287,37.94,-122.07
1130,3.0000,31.0,5.277228,1.049505,794.0,2.620462,39.76,-121.65
15026,2.6678,34.0,5.076220,1.102134,1592.0,2.426829,32.77,-117.01
4707,3.2411,52.0,5.148820,1.092559,1164.0,2.112523,34.06,-118.35


### **Train Placeholder Model** <br>
Linear Regression

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np 

lr = LinearRegression().fit(preprocessor(X_train), y_train)

# Let’s look at the training set and test set performance using r squared:
print("Training set score: {:.2f}".format(lr.score(preprocessor(X_train), y_train)))
print("Test set score: {:.2f}".format(lr.score(preprocessor(X_test), y_test)))

#cross validation
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(LinearRegression(), X_train, y_train, cv=10))

Training set score: 0.61
Test set score: 0.58


0.6112619726475526

**Save Preprocessor & Model to local files**

In [11]:
#export preprocessor
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"")

Your preprocessor is now saved to 'preprocessor.zip'


In [12]:
# Test Preprocessor
prep = ai.import_preprocessor("preprocessor.zip")
prep(example_data).shape

(5, 8)

In [13]:
# Save sklearn model to ONNX file 
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of features in preprocessed data

onnx_model = model_to_onnx(lr, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

# **Part 2: Deploy Model**

In [14]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata = example_data

In [15]:
#Deploy API
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class
myplayground=ModelPlayground(model_type="tabular", classification=False, private=False)

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)
myplayground.deploy(model_filepath, preprocessor_filepath, y_train, exampledata) 

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):CA Housing Prediction
Model Description (Explain what your model does and 
 why end-users would find your model useful):Use data from the 1990 U.S. Census to predict the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).
Model Key Words (Search categories that describe your model, separated with commas):ca housing, regression, tabular, supervised learning
   
Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 88 seconds. 
 Playground Url: "https://hcvbryu1a3.execute-api.us-east-1.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
Y

In [ ]:
#Delete Deployment if testing: 

#myplayground.delete_deployment()

Running this function will permanently delete all resources tied to this deployment, 
 including the eval lambda and all models submitted to the model competition.

To confirm, type 'permanently delete':permanently delete


'API deleted successfully.'

# **Part 3: Create a Competition**

In [16]:
# Create data directory with X_train, y_train_labels, and X_test objects: 
import os
import shutil 

X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)

os.mkdir("ca_housing_competition_data")
file_list = ["X_train.csv", "X_test.csv", "y_train.csv"]

for file in file_list: 
  shutil.move(file, "ca_housing_competition_data")

In [17]:
# Create Competition
myplayground.create_competition(data_directory='ca_housing_competition_data', 
                                y_test = y_test,  
                                public=True)


--INPUT COMPETITION DETAILS--

Enter competition name:CA Housing Prediction Competition
Enter competition description:Use data from the 1990 U.S. Census to predict the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

--INPUT DATA DETAILS--

Note: (optional) Save an optional LICENSE.txt file in your competition data directory to make users aware of any restrictions on data sharing/usage.

Enter data description (i.e.- filenames denoting training and test data, file types, and any subfolders where files are stored):Competition data folder contains labeled X_train, X_test, and y_train files.
Enter optional data license descriptive name (e.g.- 'MIT, Apache 2.0, CC0, Other, etc.'):
Uploading your data. Please wait for a confirmation message.

 Success! Model competition created. 

You may now update your prediction API runtime model and verify evaluation metrics with the update_runtime_model() function.

To upload new models and/or pre

In [18]:
#Instantiate Competition
#--Note: If you start a new session, the first argument should be the Model Playground url in quotes. 
#--e.g.- mycompetition= ai.Competition("https://2121212.execute-api.us-east-1.amazonaws.com/prod/m)
#See Model Playground "Compete" tab for example model submission code.

mycompetition= ai.Competition(myplayground.playground_url)

In [19]:
#Authorized users can submit new models after setting credentials using modelshare.org username/password
from aimodelshare.aws import set_credentials 

apiurl=myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [20]:
# Optional: Submit Model 1: 

#-- Generate predicted values (a list of predicted house values, in hundreds of thousands) (Model 1)
predicted_values = lr.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_values)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1397


Update Runtime model

*Use this function to 1) update the prediction API behind your Model Playground with a new model, chosen from the leaderboard and 2) verify the modelperformance metrics in your Model Playground*

In [21]:
import aimodelshare as ai
myplayground=ai.ModelPlayground(playground_url="https://hcvbryu1a3.execute-api.us-east-1.amazonaws.com/prod/m")
myplayground.update_runtime_model(model_version=1)

Runtime model & preprocessor for api: https://hcvbryu1a3.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 1.

Model metrics are now updated and verified for this model playground.
